In [1]:
import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

from skimage.measure import find_contours

In [3]:
polydata_list = []

In [4]:
vertices_list = []
faces_list = []

for label in ['5N']:
    for stack in ['MD589', 'MD593']:
        reader = vtk.vtkSTLReader()
        reader.SetFileName("/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s_L.stl" % \
                           {'stack':stack, 'label':label})
        reader.Update()
        
        polydata = reader.GetOutput()
        
        vertices = numpy_support.vtk_to_numpy(polydata.GetPoints().GetData())
        a = numpy_support.vtk_to_numpy(polydata.GetPolys().GetData())
        faces = np.c_[a[1::4], a[2::4], a[3::4]]
        
        vertices_list.append(vertices)
        faces_list.append(faces)
        
        reader.SetFileName("/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s_R.stl" % \
                           {'stack':stack, 'label':label})
        reader.Update()
        polydata_list.append(reader.GetOutput())
        
        polydata = reader.GetOutput()
        
        vertices = numpy_support.vtk_to_numpy(polydata.GetPoints().GetData())
        a = numpy_support.vtk_to_numpy(polydata.GetPolys().GetData())
        faces = np.c_[a[1::4], a[2::4], a[3::4]]
        
        # mirror by z axis, for some reason is the first dimension
        vertices[..., 0] = -vertices[..., 0]
        
        vertices_list.append(vertices)
        faces_list.append(faces)

In [5]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

In [6]:
from itertools import izip

In [7]:
u1 = None
vertices_normalized_aligned_list = []

for i, (vertices, faces) in enumerate(zip(vertices_list, faces_list)):
    centroid = vertices.mean(axis=0)
    scale = np.sqrt(((vertices - centroid)**2).mean())
    vertices_normalized = (vertices - centroid) / scale
    
    u, s, v = np.linalg.svd(np.dot(vertices_normalized.T, vertices_normalized)/vertices_normalized.shape[0])

    if i == 0:
        u1 = u
    
    U, _, VT = np.linalg.svd(np.dot(u1, u.T))
    R = np.dot(U, VT)

    vertices_normalized_alignedTo1 = np.dot(vertices_normalized, R.T)

    vertices_normalized_aligned_list.append(vertices_normalized_alignedTo1)

In [ ]:
polydata_list = []

for vs, fs in zip(vertices_normalized_aligned_list, faces_list):
        
    polydata = vtk.vtkPolyData()

    points = vtk.vtkPoints()
    for pt_ind, (x,y,z) in enumerate(vs):
        points.InsertPoint(pt_ind, x, y, z)

    cells = vtk.vtkCellArray()
    for ia, ib, ic in fs:
        cells.InsertNextCell(3)
        cells.InsertCellPoint(ia)
        cells.InsertCellPoint(ib)
        cells.InsertCellPoint(ic)

    polydata.SetPoints(points)
    polydata.SetPolys(cells)
    polydata.SetVerts(cells)
    
    polydata_list.append(polydata)

In [ ]:
ren1 = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren1)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

colors = [(1,0,0), (0,1,0), (0,0,1), (1,1,0)]

for i in range(len(vertices_list)):
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata_list[i])

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[i])
    
    ren1.AddActor(a)

renWin.Render()
iren.Start()

In [ ]:
def compute_vertex_normals(vertices, faces, k=20):
    ias = faces[:,0]
    ibs = faces[:,1]
    ics = faces[:,2]

    va = vertices[ias]
    vb = vertices[ibs]
    vc = vertices[ics]

    faces_normals = np.cross(va-vb, vb-vc)
#     faces_normals_normalized = faces_normals/np.sqrt(np.sum(faces_normals**2, axis=1))[:, np.newaxis]

#     adjacent_vertices = [set([]) for _  in range(len(vertices))]
#     for face_i, (ia, ib, ic) in enumerate(faces):
#         adjacent_vertices[ia].add(face_i)
#         adjacent_vertices[ib].add(face_i)
#         adjacent_vertices[ic].add(face_i)
    
#     vertices_normals = np.array([faces_normals[list(face_inds)].mean(axis=0) for face_inds in adjacent_vertices])


    face_centroids = (vertices[ias] + vertices[ibs] + vertices[ics]) / 3.
    
    face_centroid_tree = KDTree(face_centroids)
    distsToFacesClosestToVertices, indicesFacesClosestToVertices = face_centroid_tree.query(vertices, k=k)    

    vertices_normals = np.array([faces_normals[list(face_inds)].mean(axis=0) 
                                 for face_inds in indicesFacesClosestToVertices])

    
    vertices_normals_normalized = vertices/np.sqrt(np.sum(vertices**2, axis=1))[:, np.newaxis]
    
    return vertices_normals_normalized

In [ ]:
vertices1_normals_normalized = compute_vertex_normals(vertices1_normalized, faces1_corrected)
vertices2_normals_normalized = compute_vertex_normals(vertices2_normalized_alignedTo1, faces2_corrected)

In [ ]:
vertices1_normals_normalized.shape

In [ ]:
vertices2_normals_normalized.shape

In [ ]:
D = np.abs(np.dot(vertices1_normals_normalized[:100], vertices2_normals_normalized.T))

In [ ]:
ps1, ps2 = np.where(D < 1e-3)

In [ ]:
vertices2_normals_normalized[76]

In [ ]:
q = np.vstack([vertices2_normalized_alignedTo1[np.where(ps1 == i)[0]] for i in range(100)])

In [ ]:
vertices_list = [vertices1_normalized[:100], q]

In [ ]:
polydata_list = [vtk.vtkPolyData() for _ in range(2)]
for i in range(2):

    vs = vertices_list[i]
#     fs = faces_list[i]

#     vertices_vtkdata = numpy_support.numpy_to_vtk(num_array=vs, deep=True)
    
    polydata = vtk.vtkPolyData()

    points = vtk.vtkPoints()
#     points.SetData(vertices_vtkdata)
#     polydata.SetPoints(points)

    cells = vtk.vtkCellArray()
    
    c = 0
    for p in vs:
        cells.InsertNextCell(1)
        points.InsertPoint(c, p[0], p[1], p[2])
        cells.InsertCellPoint(c)
        c += 1
    
    polydata.SetPoints(points)
    polydata.SetVerts(cells)
    
    polydata_list[i].DeepCopy(polydata)
    
ren1 = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren1)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

colors = [(1,0,0), (0,1,0)]

for i in range(2):
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata_list[i])

    a = vtk.vtkActor()
    a.SetMapper(m)
#     a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[i])
    
    ren1.AddActor(a)

renWin.Render()
iren.Start()

In [ ]:
polydata_list = [vtk.vtkPolyData() for _ in range(2)]
for i in range(2):

    vs = vertices_list[i]
    fs = faces_list[i]

    vertices_vtkdata = numpy_support.numpy_to_vtk(num_array=vs, deep=True)
    
    polydata = vtk.vtkPolyData()

    points = vtk.vtkPoints()
    points.SetData(vertices_vtkdata)
    polydata.SetPoints(points)

    cells = vtk.vtkCellArray()

    tris = []
    for a,b,c in fs:
        tris.append(3)
        tris.append(a)
        tris.append(b)
        tris.append(c)
    tris = np.array(tris)

    cells.SetCells(len(tris) / 4, numpy_support.numpy_to_vtkIdTypeArray(tris))

    polydata.SetPolys(cells)
    
    polydata_list[i].DeepCopy(polydata)

In [ ]:
ren1 = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren1)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

colors = [(1,0,0), (0,1,0)]

for i in range(2):
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata_list[i])

    a = vtk.vtkActor()
    a.SetMapper(m)
#     a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[i])
    
    ren1.AddActor(a)

renWin.Render()
iren.Start()

In [ ]:
from scipy.spatial import KDTree

In [ ]:
tree1 = KDTree(vertices1_normalized)

In [ ]:
tree2 = KDTree(vertices2_normalized_alignedTo1)

In [ ]:
dists1_closestTo2, indices1_closestTo2 = tree1.query(vertices2_normalized_alignedTo1)

In [ ]:
dists2_closestTo1, indices2_closestTo1 = tree2.query(vertices1_normalized)

In [ ]:
matched_indices = set((i1, i2) for i2, i1 in enumerate(indices1_closestTo2)) & \
                    set((i1, i2) for i1, i2 in enumerate(indices2_closestTo1))

In [ ]:
matched_indices_arr = np.array(list(matched_indices))

In [ ]:
vertices1_reduced = vertices1_normalized[matched_indices_arr[:,0]]
vertices2_reduced = vertices2_normalized_alignedTo1[matched_indices_arr[:,1]]

In [ ]:
vertices_list = [vertices1_reduced, vertices2_reduced]

In [ ]:
polydata_list = [vtk.vtkPolyData() for _ in range(2)]
for i in range(2):

    vs = vertices_list[i]
#     fs = faces_list[i]

#     vertices_vtkdata = numpy_support.numpy_to_vtk(num_array=vs, deep=True)
    
    polydata = vtk.vtkPolyData()

    points = vtk.vtkPoints()
#     points.SetData(vertices_vtkdata)
#     polydata.SetPoints(points)

    cells = vtk.vtkCellArray()
    
    c = 0
    for p in vs:
        cells.InsertNextCell(1)
        points.InsertPoint(c, p[0], p[1], p[2])
        cells.InsertCellPoint(c)
        c += 1
    
    polydata.SetPoints(points)
    polydata.SetVerts(cells)
    
    polydata_list[i].DeepCopy(polydata)

In [ ]:
ren1 = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren1)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

colors = [(1,0,0), (0,1,0)]

for i in range(2):
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata_list[i])

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[i])
    
    ren1.AddActor(a)

renWin.Render()
iren.Start()